In [531]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier


import warnings
warnings.filterwarnings("ignore")

In [532]:
data = pd.read_excel("df.xlsx")
binary = {'M':1,'B':0}
data['diagnosis'] = data['diagnosis'].map(binary)

In [533]:
y = data['diagnosis']
X = data.drop(["diagnosis", "id"], axis = 1)

X_scaled = (X - np.min(X)) / (np.max(X) - np.min(X))
X = X_scaled

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2, test_size=0.2, stratify = y)

In [536]:
# Coarse
ab =  AdaBoostClassifier(DecisionTreeClassifier(random_state = 10),random_state = 10)
hp_grid_ab ={'n_estimators':[10,100,1000], 
             'base_estimator__max_depth': [5,15,25], 
             'learning_rate': [0.01, 1, 5], 
             'base_estimator__max_features': [3,12,21]}
clf_ab = GridSearchCV(ab, hp_grid_ab, cv = 5, scoring = 'recall', verbose = 1, n_jobs = -1);
clf_ab.fit(X_train, y_train);

Fitting 5 folds for each of 81 candidates, totalling 405 fits


In [537]:
clf_ab.best_params_

{'base_estimator__max_depth': 5,
 'base_estimator__max_features': 3,
 'learning_rate': 1,
 'n_estimators': 1000}

In [538]:
best_hp_ab = clf_ab.best_params_

ab_final = AdaBoostClassifier(DecisionTreeClassifier(random_state = 10, max_depth = best_hp_ab["base_estimator__max_depth"], max_leaf_nodes = best_hp_ab["base_estimator__max_features"]),random_state = 10, n_estimators = best_hp_ab["n_estimators"], learning_rate = best_hp_ab['learning_rate'])

ab_final.fit(X_train, y_train)

y_pred_ab = ab_final.predict(X_test)

print(recall_score(y_test, y_pred_ab))
print('Adaptive Boosting Tuned Confusion Matrix:\n',confusion_matrix(y_test, y_pred_ab))

0.9047619047619048
Adaptive Boosting Tuned Confusion Matrix:
 [[70  2]
 [ 4 38]]


In [553]:
# Fine
ab =  AdaBoostClassifier(DecisionTreeClassifier(random_state = 10),random_state = 10)
hp_grid_ab ={'n_estimators':np.arange(500,1500,100),
             'base_estimator__max_depth': [3,4,5,6], 
             'learning_rate': [0.5,1, 1.5], 
             'base_estimator__max_features': [2,3,4,5,6]}
clf_ab = GridSearchCV(ab, hp_grid_ab, cv = 5, scoring = 'recall', verbose = 1, n_jobs = -1);
clf_ab.fit(X_train, y_train);

Fitting 5 folds for each of 600 candidates, totalling 3000 fits


In [565]:
clf_ab.best_params_

{'base_estimator__max_depth': 3,
 'base_estimator__max_features': 4,
 'learning_rate': 1,
 'n_estimators': 500}

In [560]:
best_hp_ab = clf_ab.best_params_

ab_final = AdaBoostClassifier(DecisionTreeClassifier(random_state = 10, max_depth = best_hp_ab["base_estimator__max_depth"], max_leaf_nodes = best_hp_ab["base_estimator__max_features"]),random_state = 10, n_estimators = best_hp_ab["n_estimators"], learning_rate = best_hp_ab['learning_rate'])

ab_final.fit(X_train, y_train)

y_pred_ab = ab_final.predict(X_test)

print(recall_score(y_test, y_pred_ab))
print('Adaptive Boosting Tuned Confusion Matrix:\n',confusion_matrix(y_test, y_pred_ab))

0.9761904761904762
Adaptive Boosting Tuned Confusion Matrix:
 [[70  2]
 [ 1 41]]


In [566]:
best_model = clf_ab.best_estimator_
importances = []

for feature, importance in zip(X_train.columns, best_model.feature_importances_):
    importances.append(importance)

In [567]:
data = {"Predictors": X_train.columns, "Importances": importances}
pd.DataFrame(data).sort_values(by = 'Importances', ascending=False)

,Predictors,Importances
21,texture_worst,0.069369
1,Texture_mean,0.049403
23,area_worst,0.045258
27,concave points_worst,0.041571
0,Radius_mean,0.038130
13,area_se,0.037094
7,concave points_mean,0.036402
24,smoothness_worst,0.035781
22,perimeter_worst,0.035573
5,compactness_mean,0.034814


In [535]:
# base model
ab =  AdaBoostClassifier(DecisionTreeClassifier(random_state = 10),random_state = 10)
ab.fit(X_train, y_train);

y_pred_ab = ab.predict(X_test)

print(recall_score(y_test, y_pred_ab))
print('Adaptive Boosting Untuned Confusion Matrix:\n',confusion_matrix(y_test, y_pred_ab))

0.9285714285714286
Adaptive Boosting Untuned Confusion Matrix:
 [[68  4]
 [ 3 39]]


In [534]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(random_state = 10)

logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print(recall_score(y_test, y_pred))
print('Logistic Regression (Base Model) Confusion Matrix:\n',confusion_matrix(y_test, y_pred))

0.8809523809523809
Logistic Regression (Base Model) Confusion Matrix:
 [[72  0]
 [ 5 37]]
